In [2]:
import pandas as pd

### Here, I imported the first CSV file, which is meant for training purposes.

In [3]:
df = pd.read_csv("Costumer_Loan_Prediction_Train.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### After dropping the 'loan_id' column from the dataset as it was not necessary in predicting whether the customer will take a loan or not, I used the info() and isnull().sum() functions to gain a better understanding of the given dataset.

In [4]:
df.drop("Loan_ID", axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             601 non-null    object 
 1   Married            611 non-null    object 
 2   Dependents         599 non-null    object 
 3   Education          614 non-null    object 
 4   Self_Employed      582 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         592 non-null    float64
 8   Loan_Amount_Term   600 non-null    float64
 9   Credit_History     564 non-null    float64
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 57.7+ KB


In [5]:
df.isnull().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

### I first examined the unique values in the dataset and then utilized the mode function to fill the null values.

In [6]:
l = ["ApplicantIncome","LoanAmount","CoapplicantIncome"]
for i in df:
    if i not in l:
        print(i,"-",df[i].unique())

Gender - ['Male' 'Female' nan]
Married - ['No' 'Yes' nan]
Dependents - ['0' '1' '2' '3+' nan]
Education - ['Graduate' 'Not Graduate']
Self_Employed - ['No' 'Yes' nan]
Loan_Amount_Term - [360. 120. 240.  nan 180.  60. 300. 480.  36.  84.  12.]
Credit_History - [ 1.  0. nan]
Property_Area - ['Urban' 'Rural' 'Semiurban']
Loan_Status - ['Y' 'N']


In [7]:
l1 = ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History', 'Loan_Amount_Term','LoanAmount']
for i in l1:
    df[i].fillna(df[i].mode()[0], inplace=True)
df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

### I used the get_dummies method from the pandas library for performing one-hot encoding on categorical variables in the dataset. One-hot encoding is a technique used to convert categorical variables into binary vectors, which can be more easily used in machine learning algorithms.

In [8]:
df1 = pd.get_dummies(data = df , columns = ["Gender", "Married", "Education", "Self_Employed", "Property_Area", "Dependents"], drop_first = True)
df1.head(5)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Male,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Dependents_1,Dependents_2,Dependents_3+
0,5849,0.0,120.0,360.0,1.0,Y,1,0,0,0,0,1,0,0,0
1,4583,1508.0,128.0,360.0,1.0,N,1,1,0,0,0,0,1,0,0
2,3000,0.0,66.0,360.0,1.0,Y,1,1,0,1,0,1,0,0,0
3,2583,2358.0,120.0,360.0,1.0,Y,1,1,1,0,0,1,0,0,0
4,6000,0.0,141.0,360.0,1.0,Y,1,0,0,0,0,1,0,0,0


In [9]:
df1["Loan_Status"].replace({'Y':1,'N':0},inplace=True)

### I used the MinMaxScaler to scale down columns where necessary, and then divided the data into input (X) and output (y) variables.

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
l2 = ["ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Loan_Amount_Term"]
for i in l2:
    df1[i] = scaler.fit_transform(df1[[i]])

In [13]:
X = df1.drop('Loan_Status',axis = 1)
y = df1.Loan_Status

### I split the dataset into training and testing sets using the train_test_split function. Then, I experimented with three different algorithms - logistic regression, multinomial Naive Bayes, and Bernoulli Naive Bayes - to determine which one had better accuracy in predicting the target variable.

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
Lr = LogisticRegression(max_iter=10000)
Lr.fit(X_train, y_train)
Lr.score(X_test, y_test)

0.7886178861788617

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [19]:
multi = MultinomialNB()
multi.fit(X_train, y_train)
multi.score(X_test, y_test)

0.6585365853658537

In [20]:
from sklearn.naive_bayes import BernoulliNB

In [21]:
berno = BernoulliNB()
berno.fit(X_train, y_train)
berno.score(X_test, y_test)

0.7967479674796748

### After experimenting with the three different algorithms, I concluded that the Bernoulli Naive Bayes algorithm provided the best accuracy for predicting the target variable. I then utilized the GridSearchCV function to fine-tune the hyperparameters of the model and improve its accuracy.

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],
    'binarize': [0.0, 0.5, 1.0]
}

In [24]:
grid_search = GridSearchCV(berno, param_grid, cv=5)

In [25]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=BernoulliNB(),
             param_grid={'alpha': [0.1, 0.5, 1.0, 2.0],
                         'binarize': [0.0, 0.5, 1.0]})

In [26]:
grid_search.best_params_

{'alpha': 0.1, 'binarize': 0.0}

In [27]:
berno1 = BernoulliNB(alpha = grid_search.best_params_["alpha"], binarize = grid_search.best_params_["binarize"])
berno1.fit(X_train, y_train)
berno1.score(X_test, y_test)

0.7967479674796748

### Here, I imported the second CSV file, which is meant for testing purposes. I applied the same methods that I used on the first file, which was the training file, to obtain the input data that I can use to generate the corresponding output.

In [28]:
test = pd.read_csv("Costumer_Loan_Prediction_Test.csv")
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [29]:
test0 = test.drop("Loan_ID", axis = 1)
test0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             356 non-null    object 
 1   Married            367 non-null    object 
 2   Dependents         357 non-null    object 
 3   Education          367 non-null    object 
 4   Self_Employed      344 non-null    object 
 5   ApplicantIncome    367 non-null    int64  
 6   CoapplicantIncome  367 non-null    int64  
 7   LoanAmount         362 non-null    float64
 8   Loan_Amount_Term   361 non-null    float64
 9   Credit_History     338 non-null    float64
 10  Property_Area      367 non-null    object 
dtypes: float64(3), int64(2), object(6)
memory usage: 31.7+ KB


In [30]:
for i in l1:
    test0[i].fillna(test[i].mode()[0], inplace=True)
test0.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [31]:
test1 = pd.get_dummies(data = test0 , columns = ["Gender", "Married", "Education", "Self_Employed", "Property_Area", "Dependents"], drop_first = True)
test1.head(5)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Dependents_1,Dependents_2,Dependents_3+
0,5720,0,110.0,360.0,1.0,1,1,0,0,0,1,0,0,0
1,3076,1500,126.0,360.0,1.0,1,1,0,0,0,1,1,0,0
2,5000,1800,208.0,360.0,1.0,1,1,0,0,0,1,0,1,0
3,2340,2546,100.0,360.0,1.0,1,1,0,0,0,1,0,1,0
4,3276,0,78.0,360.0,1.0,1,0,1,0,0,1,0,0,0


In [32]:
for i in l2:
    test1[i] = scaler.fit_transform(test1[[i]])

### Finally, I made predictions for the test file using the input data obtained from the testing file, and added the predicted output to the original test file.

In [33]:
Pred = berno1.predict(test1)
Pred

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

In [34]:
test["Loan_Status"] = Pred

In [35]:
test["Loan_Status"].replace({1:'Y',0:'N'},inplace=True)

In [36]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban,Y
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban,Y
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban,Y
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban,Y
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban,Y


In [37]:
test.to_csv("Costumer_Loan_Prediction_Test_Final.csv", index = False)